# Health Diagnosis and validation agent using Autogen.
**AutoGen supports conversable agents powered by LLMs, tools, or humans, performing tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.**

In [1]:
import json
import os
import chromadb
import autogen
import pandas as pd
from autogen import ConversableAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# API Setup

In [2]:
config_list = [{"model": "llama3.1","api_type":"ollama"}]
llm_config={
    "timeout":600,
    "cache_seed":42,
    "temperature":0.3,
    "config_list":config_list,}

In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE"==str(x.get("content", ""))[-9:].upper()

In [4]:
URL = "https://pubmed.ncbi.nlm.nih.gov/"

# Convert full excel content to plain text
def extract_text_from_excel(excel_path):
    df = pd.read_excel(excel_path)
    return df.to_string(index=False)

internal_treatment_data = extract_text_from_excel("symptoms_and_treatment_policies.xlsx")

# Construct Agent

In [5]:
Symptoms_agent = ConversableAgent(
    name="Symptoms_agent",  # Removed spaces
    system_message='''You are a helpful Symptoms agent.
    You are here to get patients symptoms and their information.
    Your job is to gather patient's informations like Name, Age, Gender, health symptoms and how long they are having the symptoms.
    (e.g., {
    "Name": "Kumar",
    "Age": "30",
    "Gender": "Male"
    "symptoms": "cold, cough, Fever"
    "total symptom days":"4 days or few weeks"})
    Do not ask for other information.''',
    is_termination_msg=termination_msg,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)

Diagnose_Agent = RetrieveUserProxyAgent(
    name="Diagnose_Agent",
    human_input_mode="NEVER",
    llm_config=llm_config,
    system_message=f"You are Healthcare diagnose assistant who has extra content retrieval power for generating diagnosis for symptoms using {internal_treatment_data}",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": ["symptoms_and_treatment_policies.xlsx"],
        "docs_content": internal_treatment_data,  # Provide the extracted content directly
        "chunk_token_size": 100,
        "model": config_list[0]["model"],
        "vector_db": "chromadb",
        "get_or_create": True,
    },
    code_execution_config=False,
)

Validator_Agent = RetrieveUserProxyAgent(
    name="Validator_Agent",
    human_input_mode="NEVER",
    is_termination_msg=termination_msg,
    llm_config=llm_config,
    system_message=f"you are Healthcare validating assistant with extra content retrieval power for validating the diagnosis generated by Diagnose agent from {URL}",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": URL,
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "vector_db": "chromadb",
        "get_or_create": True,
    },
    code_execution_config=False,
)

Health_Assistant_agent = ConversableAgent(
    name="Health_Assistant_agent",  # Removed spaces
    system_message='''You are a helpful health assistant. Gather information from all agents and give the final diagnose for the symptom.''',
    is_termination_msg=termination_msg,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=termination_msg,
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)


In [6]:
chats = [
    {
        "sender": Symptoms_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get your symptoms. Could you tell me your Name, Age, Gender, health symptoms and how long they are having the symptoms ?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the patient information as JSON object only: {'Name': '', 'Age': '', 'Gender': '', 'Symptoms':'', 'total symptom days':''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": customer_proxy_agent,
        "recipient": Diagnose_Agent,
        "message": "Aggregate the details from Symptoms_agent and generate diagnosis. Do not generate new symptoms or your own symptoms.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
    {
        "sender": Diagnose_Agent,
        "recipient": Validator_Agent,
        "message": "Aggregate the details from Diagnose_Agent and validate the diagnosis. Provide accuracy rating of the diagnosis using llm intelligence. Accuracy 1 to 5, where 1 is low and 5 high.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
    {
        "sender": Validator_Agent,
        "recipient": Health_Assistant_agent,
        "message": "Aggregate the validated diagnose from Validator_Agent and reply to patient in respectful way to book appointment for physical examination and make them feel positive.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
]

# Intiating Chat

In [7]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Symptoms_agent (to customer_proxy_agent):

Hello, I'm here to help you get your symptoms. Could you tell me your Name, Age, Gender, health symptoms and how long they are having the symptoms ?

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Symptoms_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Hari, 36, Male, Frequent headache, Few weeks


customer_proxy_agent (to Symptoms_agent):

Hari, 36, Male, Frequent headache, Few weeks

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (aa7aec62-f3ae-4423-9452-d543a5a13903): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Diagnose_Agent):

Aggregate the details from Symptoms_agent and generate diagnosis. Do not generate new symptoms or your own symptoms.
Context: 
```json
{
  "Name": "Hari",
  "Age": "36",
  "Gender": "Male",
  "Symptoms": "Frequent headache",
  "total_symptom_days": "Few weeks"
}
```

Note: For the total number of symptom days, I used "Few weeks" as it's not a specific numerical value. If you have more information on this, please let me know so I can update it accurately!

-----------------------------------------------